In [98]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

<p>Busca trending<p>

In [99]:
spark = SparkSession.builder.appName('Exploracao').getOrCreate()

In [100]:
df__video_json = spark.read.json('../data/projeto_youtube/bronze/cities_skylines/extracao_data_2023_10_11/estatisticas/*.json')

In [101]:
df__video_json.show(truncate=False)

+-------------------+---------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [102]:
df__video_json = df__video_json.select(
    'data_extracao',
      f.explode('items').alias('items')

      )

In [103]:
df__video_json.printSchema()

root
 |-- data_extracao: string (nullable = true)
 |-- items: struct (nullable = true)
 |    |-- contentDetails: struct (nullable = true)
 |    |    |-- caption: string (nullable = true)
 |    |    |-- definition: string (nullable = true)
 |    |    |-- dimension: string (nullable = true)
 |    |    |-- duration: string (nullable = true)
 |    |    |-- licensedContent: boolean (nullable = true)
 |    |    |-- projection: string (nullable = true)
 |    |    |-- regionRestriction: struct (nullable = true)
 |    |    |    |-- blocked: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |-- etag: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- kind: string (nullable = true)
 |    |-- snippet: struct (nullable = true)
 |    |    |-- categoryId: string (nullable = true)
 |    |    |-- channelId: string (nullable = true)
 |    |    |-- channelTitle: string (nullable = true)
 |    |    |-- defaultAudioLanguage: string (nullable

In [104]:
df__video_json.select(
    'data_extracao',
    f.col('items.id').alias('ID_VIDEO'),
    f.col('items.snippet.publishedAt').alias('DATA_PUBLICACAO'),
    f.col('items.snippet.channelId').alias('ID_CANAL'),
    f.col('items.snippet.channelTitle').alias('NM_CANAL'),
    f.col('items.snippet.tags').alias('TAGS'),
    f.col('items.contentDetails.duration').alias('DURACAO_VIDEOS'),
    f.col('items.statistics.viewCount').alias('TOTAL_VIDEOS'),
    f.col('items.statistics.likeCount').alias('TOTAL_LIKES'),
    f.col('items.statistics.favoriteCount').alias('TOTAL_FAVORITOS'),
    f.col('items.statistics.commentCount').alias('TOTAL_COMENTARIOS'),
).show()

+-------------------+-----------+--------------------+--------------------+-----------+--------------------+--------------+------------+-----------+---------------+-----------------+
|      data_extracao|   ID_VIDEO|     DATA_PUBLICACAO|            ID_CANAL|   NM_CANAL|                TAGS|DURACAO_VIDEOS|TOTAL_VIDEOS|TOTAL_LIKES|TOTAL_FAVORITOS|TOTAL_COMENTARIOS|
+-------------------+-----------+--------------------+--------------------+-----------+--------------------+--------------+------------+-----------+---------------+-----------------+
|2023-10-11 19:23:42|up9HGPZ9bYc|2023-10-11T19:17:54Z|UCHm9aiQsllQbGcxu...|  Fumacinha|[mta highlights, ...|        PT2M6S|          64|         10|              0|                3|
|2023-10-11 19:23:42|G022-0aKu18|2023-10-11T21:22:54Z|UCMsbzwrSvRJoa0Jh...|  rlkpedrom|                null|     PT3H52M9S|         179|          9|              0|                0|
|2023-10-11 19:23:42|nxmL--7BKhA|2023-10-11T19:44:38Z|UCIQpH9j6NIgeO481...|    eazy c

In [167]:
df_comentarios_json = spark.read.json('../data/projeto_youtube/bronze/cities_skylines/extracao_data_2023_10_11/comentarios/*.json')

In [168]:
df_comentarios_json.printSchema()

root
 |-- data_extracao: string (nullable = true)
 |-- etag: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- etag: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- kind: string (nullable = true)
 |    |    |-- snippet: struct (nullable = true)
 |    |    |    |-- canReply: boolean (nullable = true)
 |    |    |    |-- channelId: string (nullable = true)
 |    |    |    |-- isPublic: boolean (nullable = true)
 |    |    |    |-- topLevelComment: struct (nullable = true)
 |    |    |    |    |-- etag: string (nullable = true)
 |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |-- kind: string (nullable = true)
 |    |    |    |    |-- snippet: struct (nullable = true)
 |    |    |    |    |    |-- authorChannelId: struct (nullable = true)
 |    |    |    |    |    |    |-- value: string (nullable = true)
 |    |    |    |    |    |-- authorChannelUrl: str

In [182]:
df_comentarios_json.select(
    'data_extracao',
    f.explode('items').alias('ITEMS'),

) \
    .select('ITEMS.*').printSchema()


root
 |-- etag: string (nullable = true)
 |-- id: string (nullable = true)
 |-- kind: string (nullable = true)
 |-- snippet: struct (nullable = true)
 |    |-- canReply: boolean (nullable = true)
 |    |-- channelId: string (nullable = true)
 |    |-- isPublic: boolean (nullable = true)
 |    |-- topLevelComment: struct (nullable = true)
 |    |    |-- etag: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- kind: string (nullable = true)
 |    |    |-- snippet: struct (nullable = true)
 |    |    |    |-- authorChannelId: struct (nullable = true)
 |    |    |    |    |-- value: string (nullable = true)
 |    |    |    |-- authorChannelUrl: string (nullable = true)
 |    |    |    |-- authorDisplayName: string (nullable = true)
 |    |    |    |-- authorProfileImageUrl: string (nullable = true)
 |    |    |    |-- canRate: boolean (nullable = true)
 |    |    |    |-- channelId: string (nullable = true)
 |    |    |    |-- likeCount: long (nullable = tr

In [206]:
df_comentarios_json.select(
    'data_extracao',
    f.explode('items').alias('ITEMS'),

) \
    .select( 
        f.col('data_extracao').alias('DATA_EXTRACAO'), 
        f.col('ITEMS.snippet.channelId').alias('ID_CANAL'),
        f.col('ITEMS.id').alias('ID_COMENTARIO'),
        f.col('ITEMS.snippet.videoId').alias('ID_VIDEO'),
        f.col('ITEMS.snippet.topLevelComment.snippet.textDisplay').alias('TEXTO_COMENTARIO'),
        f.col('ITEMS.snippet.topLevelComment.snippet.likeCount').alias('TOTAL_LIKES'),
        f.col('ITEMS.snippet.topLevelComment.snippet.publishedAt').alias('DATA_PUBLICACAO'),
        f.col('ITEMS.snippet.topLevelComment.snippet.updatedAt').alias('DATA_PUBLICACAO'),
        f.col('ITEMS.snippet.totalReplyCount').alias('TOTAL_RESPOSTAS')
    ).show(truncate=False)


+-------------------+------------------------+--------------------------+-----------+--------------------------------------------------------------------------------------------------------------+-----------+--------------------+--------------------+---------------+
|DATA_EXTRACAO      |ID_CANAL                |ID_COMENTARIO             |ID_VIDEO   |TEXTO_COMENTARIO                                                                                              |TOTAL_LIKES|DATA_PUBLICACAO     |DATA_PUBLICACAO     |TOTAL_RESPOSTAS|
+-------------------+------------------------+--------------------------+-----------+--------------------------------------------------------------------------------------------------------------+-----------+--------------------+--------------------+---------------+
|2023-10-11 19:23:46|UCHm9aiQsllQbGcxu4ynJCAA|UgzB7rDtST0fOg8XT1B4AaABAg|up9HGPZ9bYc|brb                                                                                                           |0  

In [178]:
df_comentarios_json.select(
    'data_extracao',
    f.explode('items').alias('ITEMS'),

).printSchema()


root
 |-- data_extracao: string (nullable = true)
 |-- ITEMS: struct (nullable = true)
 |    |-- etag: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- kind: string (nullable = true)
 |    |-- snippet: struct (nullable = true)
 |    |    |-- canReply: boolean (nullable = true)
 |    |    |-- channelId: string (nullable = true)
 |    |    |-- isPublic: boolean (nullable = true)
 |    |    |-- topLevelComment: struct (nullable = true)
 |    |    |    |-- etag: string (nullable = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- kind: string (nullable = true)
 |    |    |    |-- snippet: struct (nullable = true)
 |    |    |    |    |-- authorChannelId: struct (nullable = true)
 |    |    |    |    |    |-- value: string (nullable = true)
 |    |    |    |    |-- authorChannelUrl: string (nullable = true)
 |    |    |    |    |-- authorDisplayName: string (nullable = true)
 |    |    |    |    |-- authorProfileImageUrl: string (nullable = 

In [166]:
df_comentarios_json.show()

+-------------------+--------------------+
|      data_extracao|            ID_CANAL|
+-------------------+--------------------+
|2023-10-11 19:23:46|UCHm9aiQsllQbGcxu...|
|2023-10-11 19:23:46|UCHm9aiQsllQbGcxu...|
|2023-10-11 19:23:46|UCHm9aiQsllQbGcxu...|
|2023-10-11 19:23:46|UCIQpH9j6NIgeO481...|
|2023-10-11 19:23:47|UCoBbeu5bEvrJJDCC...|
|2023-10-11 19:23:47|UCoBbeu5bEvrJJDCC...|
|2023-10-11 19:23:47|UCoBbeu5bEvrJJDCC...|
|2023-10-11 19:23:47|UCoBbeu5bEvrJJDCC...|
|2023-10-11 19:23:47|UCoBbeu5bEvrJJDCC...|
|2023-10-11 19:23:47|UCoBbeu5bEvrJJDCC...|
|2023-10-11 19:25:22|UCHm9aiQsllQbGcxu...|
|2023-10-11 19:25:22|UCHm9aiQsllQbGcxu...|
|2023-10-11 19:25:22|UCHm9aiQsllQbGcxu...|
|2023-10-11 19:25:23|UCIQpH9j6NIgeO481...|
|2023-10-11 19:25:23|UCoBbeu5bEvrJJDCC...|
|2023-10-11 19:25:23|UCoBbeu5bEvrJJDCC...|
|2023-10-11 19:25:23|UCoBbeu5bEvrJJDCC...|
|2023-10-11 19:25:23|UCoBbeu5bEvrJJDCC...|
|2023-10-11 19:25:23|UCoBbeu5bEvrJJDCC...|
|2023-10-11 19:25:23|UCoBbeu5bEvrJJDCC...|
+----------

In [157]:
df_comentarios_json = df_comentarios_json.select(
    'data_extracao',
    f.explode('ID_CANAL'),
    f.explode('ID_COMENTARIO')
    )

AnalysisException: Only one generator allowed per select clause but found 2: explode(ID_CANAL), explode(ID_COMENTARIO)

In [ ]:
df_comentarios_json.show(truncate=False)

+-------------------+--------------------+
|      data_extracao|                 col|
+-------------------+--------------------+
|2023-10-11 19:23:46|UgzB7rDtST0fOg8XT...|
|2023-10-11 19:23:46|Ugxpcnm0-FIUzUTe-...|
|2023-10-11 19:23:46|UgwxGST9Gz9hVbPKq...|
|2023-10-11 19:23:46|UgzB7rDtST0fOg8XT...|
|2023-10-11 19:23:46|Ugxpcnm0-FIUzUTe-...|
|2023-10-11 19:23:46|UgwxGST9Gz9hVbPKq...|
|2023-10-11 19:23:46|UgzB7rDtST0fOg8XT...|
|2023-10-11 19:23:46|Ugxpcnm0-FIUzUTe-...|
|2023-10-11 19:23:46|UgwxGST9Gz9hVbPKq...|
|2023-10-11 19:23:46|Ugzm0ZKIrLAT-yU15...|
|2023-10-11 19:23:47|UgzpWa-p3sv-b5Syt...|
|2023-10-11 19:23:47|UgxXHqIBioOEl4Iqi...|
|2023-10-11 19:23:47|UgxAR-9TFpkkLZ0HY...|
|2023-10-11 19:23:47|UgyOoa1ylH-4guduS...|
|2023-10-11 19:23:47|Ugy6pl28VfZ9PhfdB...|
|2023-10-11 19:23:47|UgzMVRuKSXLnSDLDw...|
|2023-10-11 19:23:47|UgxXHqIBioOEl4Iqi...|
|2023-10-11 19:23:47|UgxAR-9TFpkkLZ0HY...|
|2023-10-11 19:23:47|UgyOoa1ylH-4guduS...|
|2023-10-11 19:23:47|Ugy6pl28VfZ9PhfdB...|
+----------